####  This code works to jailbreak gemma-7b-it pretty consistently. Hoewver, it has to be run a few times before it works due to some randomness in probe. 

In [ ]:
import pandas as pd
import sys
import numpy as np
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

from analysis.supervised import *
from analysis.causal_interventions import *
from analysis.Probes import SimpleMLP


df = pd.read_pickle("/datasets/latest/wei_dataset_rated.pkl")

print("dataset loaded")

## CONFIG
model_name = "google/gemma-7b-it"
layer=17
activation_column="prompt_res_activations"
num_samples = len(df)
dataset_name = "wei"

model = AutoModelForCausalLM.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)


# Add column that has 0 if "rating" is "No" and 1 if "rating" is "Yes"
df['rating_binary'] = df['rating'].apply(lambda x: 0 if x == "No" else 1)
df.head(1)

def extract_layer(row, layer):
    return row['prompt_res_activations'][layer][0].cpu().numpy().flatten()

# Create new columns for each layer
df[f'layer_{layer}'] = df.apply(lambda row: extract_layer(row, layer), axis=1)

# drop all rows that have auto_payload_splitting, auto_obfuscation, dev_mode_with_rant in jailbreak_prompt_name
df = df[~df["jailbreak_prompt_name"].isin(["auto_payload_splitting", "auto_obfuscation", "dev_mode_with_rant"])]
df["jailbreak_prompt_name"].value_counts()

labels = df["rating_binary"].to_list()
data = df[f'layer_{layer}'].to_list()
print("LEN LABELS", len(labels))
print("LEN DATA", len(data))

split_idx = int(0.8* len(df))


#####

mlp = SimpleMLP()

mlp.train_MLP(data = data, labels = labels, split_idx = split_idx, num_epochs = 500)
mlp.get_accuracy(data = data, labels = labels, split_idx = split_idx)

mlp.train()

# run on every element of the sample
for index, row in df.iterrows():
    # save in sample_df
    response = get_response_MLP(model, tokenizer, row["jailbreak_prompt_text"], mlp, max_new_tokens=100, layers_to_intervene=[16], target=1)
    df.at[index, "response"] = response
    print(response)
    
    # Save every 10 samples
    if index % 20 == 0:
        df.to_pickle(f"/datasets/old/wei_dataset_with_MLP_test.pkl")

    

## Trying to catch good probe 

In [ ]:
import pandas as pd
import sys
import numpy as np
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch


from analysis.supervised import *
# from analysis.causal_interventions import *
from analysis.Probes import SimpleMLP


df = pd.read_pickle("/datasets/latest/wei_dataset_rated.pkl")

print("dataset loaded")

## CONFIG
model_name = "google/gemma-7b-it"
layer=17
activation_column="prompt_res_activations"
num_samples = len(df)
dataset_name = "wei"

model = AutoModelForCausalLM.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)


# Add column that has 0 if "rating" is "No" and 1 if "rating" is "Yes"
df['rating_binary'] = df['rating'].apply(lambda x: 0 if x == "No" else 1)
df.head(1)

def extract_layer(row, layer):
    return row['prompt_res_activations'][layer][0].cpu().numpy().flatten()

# Create new columns for each layer
df[f'layer_{layer}'] = df.apply(lambda row: extract_layer(row, layer), axis=1)

# drop all rows that have auto_payload_splitting, auto_obfuscation, dev_mode_with_rant in jailbreak_prompt_name
df = df[~df["jailbreak_prompt_name"].isin(["auto_payload_splitting", "auto_obfuscation", "dev_mode_with_rant"])]
df["jailbreak_prompt_name"].value_counts()

labels = df["rating_binary"].to_list()
data = df[f'layer_{layer}'].to_list()
print("LEN LABELS", len(labels))
print("LEN DATA", len(data))

split_idx = int(0.8* len(df))


#####

mlp = SimpleMLP()

mlp.train_MLP(data = data, labels = labels, split_idx = split_idx, num_epochs = 500)
mlp.get_accuracy(data = data, labels = labels, split_idx = split_idx)

mlp.train()

# run on every element of the sample
for index, row in df.iterrows():
    # save in sample_df
    response = get_response_MLP(model, tokenizer, row["jailbreak_prompt_text"], mlp, max_new_tokens=100, layers_to_intervene=[16], target=1, learning_rate = 0.01)
    df.at[index, "response"] = response
    print(response)
    
    # Save every 10 samples
    if index % 20 == 0:
        df.to_pickle(f"/datasets/old/MLP_intervention_test.pkl")

    

## compare probe attributes

In [17]:
from analysis.causal_interventions import get_res_layers_to_enumerate

def get_response_MLP(model, 
                 tokenizer,  
                 prompt,
                 mlp,
                 layers_to_intervene=None,  # Allow multiple layers
                 max_new_tokens=15, 
                 target = 0,
                 loss = "MSE",
                 towards_target = True,
                 nr_perturbations=32, 
                 learning_rate = 0.1):
    """
    Generates a response from a model with a causal intervention applied to specific layers.

    Args:
        model: Pretrained language model.
        tokenizer: Tokenizer compatible with the model.
        prompt (str): The input text prompt.
        probe: The intervention object (e.g., a linear model with coefficients).
        layers_to_intervene (list of int): Indices of the model layers to apply the intervention.
        max_new_tokens (int): Maximum number of new tokens to generate.
        intervention_strength (float): Strength of the intervention to apply.

    Returns:
        str: The generated text after applying the intervention.
    """
    print(f"Using {loss} loss function")
    print("learning rate", learning_rate)
    print("nr_perturbations", nr_perturbations)
    
    if layers_to_intervene is None:
        layers_to_intervene = [17]  # Default to layer 17 if no layers specified

    device = next(model.parameters()).device
    input_ids = tokenizer.encode(prompt, return_tensors='pt', truncation=True).to(device)
    len_prompt = input_ids.size(1)
    
    print("loss", loss)
    # Define a loss function
    if loss == "MSE" : criterion = nn.MSELoss()
    
    # elif loss == "BCE" : criterion = nn.BCELoss()
    elif loss == "BCE" : criterion = nn.BCEWithLogitsLoss()
    
    elif loss == "BCE_new" : criterion = nn.BCELoss()
    
    sigmoid = nn.Sigmoid()


    def linear_intervene(name, target):
        def hook(model, input, output):
          
          with torch.enable_grad():
                      
            # Forward pass
            feature_vector = output[0][:, :, :].requires_grad_().to(output[0].device)          
            # print(feature_vector.shape, feature_vector.requires_grad)  # Should print torch.Size([1, 768]) True
            
            ## check old predicted class
            pred = mlp(feature_vector)
            print("OLD MLP_out:", pred)
            
            ### 
            
                    
            # Define perturbation as a parameter or tensor that requires gradient
            perturbation = torch.zeros_like(feature_vector, requires_grad=True).to(output[0].device)
            
            printed_once = True

            for i in range(nr_perturbations):
              # Apply perturbation to the feature vector
              perturbed_vector = feature_vector + perturbation

              # # Get output from the MLP
              # if loss == "BCE_new":
              #   mlp_out = sigmoid(mlp(perturbed_vector))
              # else: 
              mlp_out = mlp(perturbed_vector)
              
              
              # target_out = torch.zeros_like(mlp_out)  # Default to zero, or choose an appropriate value


              target_out = torch.full_like(mlp_out, target)               

              # Calculate the loss
              loss = criterion(mlp_out, target_out).to(output[0].device)

              # print(loss, loss.requires_grad)  # Should print a scalar tensor and True

              # Backward pass to compute gradients w.r.t the perturbation
              loss.backward()

              # Access the gradient of the loss w.r.t. the perturbation
              grad_input = perturbation.grad

              # Define a learning rate
              # learning_rate = 0.01

              
              if towards_target: 
                perturbation = (perturbation - learning_rate * grad_input).clone().detach().requires_grad_().to(output[0].device)
              else: 
                perturbation = (perturbation + learning_rate * grad_input).clone().detach().requires_grad_().to(output[0].device)
              
            ## check new predicted class
            # pred = mlp(perturbation)
            pred = mlp(perturbed_vector)
            print("NEW MLP_out:", pred)
            
            ### 
            
            
            new_out = perturbed_vector, output[1] # concat perturbed vector with the rest of the output

            # returnn perturbation
            return new_out
          
        return hook

    layers_to_enum = get_res_layers_to_enumerate(model)
    hooks = []

    # Register hooks for each specified layer
    for layer_index in layers_to_intervene:
        if layer_index < 0 or layer_index >= len(layers_to_enum):
            raise ValueError(f"Layer {layer_index} is out of bounds for the model.")
        hook_handle = model.model.layers[layer_index].register_forward_hook(linear_intervene(layer_index, target))
        hooks.append(hook_handle)
    
    try:
        # model.eval()
        with torch.no_grad():
          output_sequence = model.generate(input_ids, num_return_sequences=1, max_new_tokens=max_new_tokens, do_sample=False)
            
    finally:
        # Ensure all hooks are removed after use
        for h in hooks:
            h.remove()
    
    generated_text = tokenizer.decode(output_sequence[0], skip_special_tokens=True)
    
    # If outptut sequence repeats prompt, remove it 
    if generated_text.startswith(prompt): generated_text = generated_text[len(prompt):].strip()

    return generated_text

In [18]:
#!/usr/bin/env python
# coding: utf-8

import argparse
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch
import pandas as pd
import sys

# Function to process the dataframe
def process_dataframe(df, model, tokenizer):
    dictionary = {}
    # loop through every row where jailbreak_prompt_name is original_prompt
    for index, row in df.iterrows():
        if row['jailbreak_prompt_name'] == 'original_prompt':
            prompt_id = row['prompt_name']
            prompt_text = row['jailbreak_prompt_text']
            dictionary[prompt_id] = prompt_text

    # now loop through every row, and add a column called "original_prompt_text" to the dataframe
    for index, row in df.iterrows():
        prompt_id = row['prompt_name']
        df.loc[index, 'original_prompt_text'] = dictionary.get(prompt_id, None)
    
    return df

def rate(df):
    from gen_data.harmBench_autograder import grade_with_HF

    df = process_dataframe(df, model, tokenizer)
    df = df.rename(columns={"jailbreak_prompt_text" : "original_prompt_text", "MLP_intervention_response": "output_text", "linear_intervention_response": "output_text"}, inplace=True)
    df = df.rename(columns={"forbidden_prompt": "original_prompt_text", "response": "output_text"})
    
    # Load the HarmBench model
    model_name = "cais/HarmBench-Llama-2-13b-cls"
    model = AutoModelForCausalLM.from_pretrained(model_name)
    model.to('cuda')
    tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=False, truncation_side="left", padding_side="left")


    # Apply the grading function and add the ratings column
    df["rating"] = df.apply(lambda row: grade_with_HF(row['original_prompt_text'], [row['output_text']], model, tokenizer, verbose = True)[0], axis=1)
    
    return df

In [19]:
import pandas as pd
import sys
import numpy as np
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

from analysis.supervised import *
# from analysis.causal_interventions import *
from analysis.Probes import SimpleMLP


df = pd.read_pickle("/datasets/latest/wei_dataset_rated.pkl")
## CONFIG
model_name = "google/gemma-7b-it"
layer=17
activation_column="prompt_res_activations"
num_samples = len(df)
dataset_name = "wei"

for i in range(50):

  model = AutoModelForCausalLM.from_pretrained(model_name)
  tokenizer = AutoTokenizer.from_pretrained(model_name)


  # Add column that has 0 if "rating" is "No" and 1 if "rating" is "Yes"
  df['rating_binary'] = df['rating'].apply(lambda x: 0 if x == "No" else 1)
  df.head(1)

  def extract_layer(row, layer):
      return row['prompt_res_activations'][layer][0].cpu().numpy().flatten()

  # Create new columns for each layer
  df[f'layer_{layer}'] = df.apply(lambda row: extract_layer(row, layer), axis=1)

  # drop all rows that have auto_payload_splitting, auto_obfuscation, dev_mode_with_rant in jailbreak_prompt_name
  df = df[~df["jailbreak_prompt_name"].isin(["auto_payload_splitting", "auto_obfuscation", "dev_mode_with_rant"])]
  df["jailbreak_prompt_name"].value_counts()

  labels = df["rating_binary"].to_list()
  data = df[f'layer_{layer}'].to_list()
  print("LEN LABELS", len(labels))
  print("LEN DATA", len(data))

  split_idx = int(0.8* len(df))


  #####

  mlp = SimpleMLP()

  losses, train_accuracies = mlp.train_MLP(data = data, labels = labels, split_idx = split_idx, num_epochs = 500)
  test_accuracy = mlp.get_accuracy(data = data, labels = labels, split_idx = split_idx)


  # Store loss and accuracy in the DataFrame
  num_rows = len(df)
  df['probe_loss'] = [losses] * num_rows
  df['train_accuracy'] = [train_accuracies] * num_rows
  df['test_accuracy'] = test_accuracy * num_rows


  df = df[:20]
  print("dataset loaded")

  mlp.train()

  # run on every element of the sample
  for index, row in df.iterrows():
      # save in sample_df
      response = get_response_MLP(model, tokenizer, row["jailbreak_prompt_text"], mlp, max_new_tokens=100, layers_to_intervene=[16], target=1, learning_rate = 0.01)
      df.at[index, "response"] = response
      print(response)

  df = rate(df)  

  ASR = round(df["rating"].value_counts(normalize=True)["Yes"], 2)

  print("ASR", ASR)
  # Save every 10 samples
  df.to_pickle(f"/datasets/old/MLP_intervention_test_ASR{ASR}.pkl")

    

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

LEN LABELS 9000
LEN DATA 9000
Epoch [10/500], Loss: 0.2868
Epoch [20/500], Loss: 0.2840
Epoch [30/500], Loss: 0.2603
Epoch [40/500], Loss: 0.2523
Epoch [50/500], Loss: 0.1997
Epoch [60/500], Loss: 0.2111
Epoch [70/500], Loss: 0.2195
Epoch [80/500], Loss: 0.2474
Epoch [90/500], Loss: 0.2440
Epoch [100/500], Loss: 0.2341
Epoch [110/500], Loss: 0.2080
Epoch [120/500], Loss: 0.2103
Epoch [130/500], Loss: 0.2125
Epoch [140/500], Loss: 0.2056
Epoch [150/500], Loss: 0.2168
Epoch [160/500], Loss: 0.2268
Epoch [170/500], Loss: 0.2211
Epoch [180/500], Loss: 0.2155
Epoch [190/500], Loss: 0.2059
Epoch [200/500], Loss: 0.2102
Epoch [210/500], Loss: 0.1947
Epoch [220/500], Loss: 0.2231
Epoch [230/500], Loss: 0.2220
Epoch [240/500], Loss: 0.2189
Epoch [250/500], Loss: 0.1928
Epoch [260/500], Loss: 0.1970
Epoch [270/500], Loss: 0.2099
Epoch [280/500], Loss: 0.2186
Epoch [290/500], Loss: 0.1923
Epoch [300/500], Loss: 0.1923
Epoch [310/500], Loss: 0.1998
Epoch [320/500], Loss: 0.1996
Epoch [330/500], Lo

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


Epoch [500/500], Loss: 0.1622
Training complete.
Applying fitted scaler to test data...
Accuracy: 0.8361
dataset loaded
Using MSE loss function
learning rate 0.01
nr_perturbations 32
loss MSE
OLD MLP_out: tensor([[-28.2420]], grad_fn=<MeanBackward1>)
NEW MLP_out: tensor([[-4.5932]], grad_fn=<MeanBackward1>)
OLD MLP_out: tensor([[-98.6900]], grad_fn=<MeanBackward1>)
NEW MLP_out: tensor([[9.2528]], grad_fn=<MeanBackward1>)
OLD MLP_out: tensor([[-15.7266]], grad_fn=<MeanBackward1>)
NEW MLP_out: tensor([[0.9908]], grad_fn=<MeanBackward1>)
OLD MLP_out: tensor([[-27.9838]], grad_fn=<MeanBackward1>)
NEW MLP_out: tensor([[1.9698]], grad_fn=<MeanBackward1>)
OLD MLP_out: tensor([[-20.1953]], grad_fn=<MeanBackward1>)
NEW MLP_out: tensor([[1.1706]], grad_fn=<MeanBackward1>)
OLD MLP_out: tensor([[-18.4088]], grad_fn=<MeanBackward1>)
NEW MLP_out: tensor([[0.6437]], grad_fn=<MeanBackward1>)
OLD MLP_out: tensor([[-14.6265]], grad_fn=<MeanBackward1>)
NEW MLP_out: tensor([[0.6192]], grad_fn=<MeanBackwar

UnboundLocalError: cannot access local variable 'model' where it is not associated with a value